In [1]:
import numpy as np
from math import comb
from docplex.mp import model

# Data generation


In [72]:
class DataGeneration:
    """Args:
    n: int - number of vehicle types
    m: int - number of time periods
    pm: int - max number of vehicles for total
    """
    def __init__(self, n: int, m: int, pm: int, random_state: int) -> None:

        self.n = n
        self.m = m
        self.pm = pm
        self.random_state = random_state


    def get_parameters(self):
        """Generate parameters.

        Returns:
            alpha: list of random integers between [1, 1000]
            beta: list of random integers between [1, 1000]
            gamma: list of random integers between [alpha[i]+beta[i], 2*(alpha[i]+beta[i])]
            pmax: list of maximum number of vehicles of each type are randon integer between [1,pm]
            rho: n x m array of random floats between [0, 1]
            c: list of random integers between [1, 100] (cost of each type of vehicle)
            b: capacity
        """
        np.random.seed(self.random_state)
        alpha = np.random.randint(2, 100, size=self.n)
        beta = np.random.randint(2, 100, size=self.n)
        gamma = np.random.randint((alpha + beta)+1, 2 * (alpha + beta))
        pmax = np.random.randint(int(self.pm/2), self.pm , size=self.n)
        rho = np.random.rand(self.n, self.m)
        c = np.random.randint(2, 10, size=self.n)
        b_min = int(np.sum(c*pmax)/self.n)
        b = np.random.randint(b_min, int(2*b_min))

        return alpha, beta, gamma, pmax, rho, c, b
    
    def get_probability(self, pimax: int, pij: float) -> float:
        """Generate probability.

        Args:
            pimax: int - maximum number of vehicles
            pij: float in [0, 1] - parameter for distribution

        Returns:
            float in (0, 1) - probability at time j, there are k vehicles of type i required
        """
        assert 0 <= pij <= 1

        return np.array([comb(pimax, k) * pij**k * (1-pij)**(pimax - k) for k in range(pimax)])



# Algorithms design
1. standard cutting plane approach
2. linearization approach

In [134]:
class Algorithms:
    """Args:
    data: input data for the models
    timeout: time limits for each models, default 200s
    """
    def __init__(self, data, timeout: float = 200) -> None:
        paramter = data.get_parameters()
        rho = paramter[4]
        pmax = paramter[3]
        self.data = data
        self.m = data.m
        self.n = data.n
        self.pmax = pmax
        self.c = paramter[5]
        self.b = paramter[6]
        self.alpha = paramter[0]
        self.beta = paramter[1]
        self.gamma = paramter[2]
        self.rho = rho
        self.timeout = timeout
        self.prob = [[data.get_probability(pmax[i],rho[i][j]) 
                      for j in range(data.m)] 
                      for i in range(data.n)]

    def get_value_fi(self, xi: float, i: int) -> float:

        beta_i = self.beta[i]     #int
        gamma_i = self.gamma[i]   #int
        prob_i = self.prob[i]     #np.array mxpmax[i]

        # get cost for each stage j
        value_j = np.array([np.sum(np.array([ prob_i[j][k] 
                                            *(beta_i * min(k, xi) 
                                            + gamma_i * max(k - xi, 0)) 
                                            for k in range(self.pmax[i])]))
                                            for j in range(self.m)])
    
            
                
        return np.sum(value_j)
    
    def get_value(self, x: np.array) -> float:
        """Generate cost value

        Args:
            x: current selection

        Returns:
            float: cost of maintenance and hiring more vehicles
        """
        
        return np.sum(np.array([self.get_value_fi(x[i], i)
                        for i in range(self.n)]))

    
    def get_sub_gradient(self, x: np.array) -> np.array:
        """Generate subgradient

        Args:
            x: current selection

        Returns:
            np.array: an array of subgradient vector at x
        """
        subgrad = np.zeros(self.n)
        for i in range(self.n):
            if x[i].is_integer():
                
                if x[i] < 1:
                    subgrad[i] = self.get_value_fi(x[i]+1,i) - self.get_value_fi(x[i],i)
                else:
                    subgrad[i] = 1/2*(self.get_value_fi(x[i]+1,i) - self.get_value_fi(x[i]-1,i))
            else:
                subgrad[i] = self.get_value_fi(np.ceil(x[i]),i) - self.get_value_fi(np.floor(x[i]),i)

        return subgrad




    def linearization(self):

        # set up linear model
        ml = model.Model(log_output=False)
        ml.parameters.timelimit = self.timeout

        # set variables
        x = ml.integer_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set auxiliary variables 
        ij = [(i,j,k) for i in range(self.n) 
                      for j in range(self.m)
                      for k in range(self.pmax[i])]
        
        y = ml.continuous_var_dict(ij, name = "y")

        # set up constraints
        ml.add_constraint(ml.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # add y constraints
        ml.add_constraints(y[i,j,k] >= k*self.beta[i] for (i,j,k) in ij)
        ml.add_constraints(y[i,j,k] >= self.beta[i]*x[i] + self.gamma[i]* (k - x[i])
                          for (i,j,k) in ij)

        # objective function
        ml.minimize(ml.sum(self.m*self.alpha[i]*x[i] 
                            + ml.sum(self.prob[i][j][k]*y[i,j,k] 
                                 for j in range(self.m) 
                                 for k in range(self.pmax[i]))  
                                 for i in range(self.n)))

        ml.solve()

        return ml.objective_value, ml.solve_details.time, ml.solve_details.mip_relative_gap
    
    def linearization_relaxed(self):

        # set up linear model
        ml = model.Model(log_output=False)
        ml.parameters.timelimit = self.timeout

        # set variables
        x = ml.continuous_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set auxiliary variables 
        ij = [(i,j,k) for i in range(self.n) 
                      for j in range(self.m)
                      for k in range(self.pmax[i])]
        
        y = ml.continuous_var_dict(ij, name = "y")

        # set up constraints
        ml.add_constraint(ml.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # add y constraints
        ml.add_constraints(y[i,j,k] >= k*self.beta[i] for (i,j,k) in ij)
        ml.add_constraints(y[i,j,k] >= self.beta[i]*x[i] + self.gamma[i]* (k - x[i])
                          for (i,j,k) in ij)

        # objective function
        ml.minimize(ml.sum(self.m*self.alpha[i]*x[i] 
                            + ml.sum(self.prob[i][j][k]*y[i,j,k] 
                                 for j in range(self.m) 
                                 for k in range(self.pmax[i]))  
                                 for i in range(self.n)))

        ml.solve()

        return ml.objective_value, ml.solve_details.time, ml.solve_details.mip_relative_gap


    def cutting_plane(self,iter_max: int = 200, tol: float = 1e-4):

        # set up linear model
        m = model.Model(log_output=False)
        m.parameters.timelimit = self.timeout

        # set variables
        x = m.integer_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set up constraints
        m.add_constraint(m.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # get initial solution
        m.minimize(m.sum(self.alpha[i]*x[i] for i in range(self.n)))
        
        m.solve()
        xk = np.array(m.solution.get_value_list(x))
        
        # set upperbound
        lower = 0
        upper = lower + 1
        num_iter = 0
        time_solve = 0

        # set theta
        theta = m.continuous_var(name="theta")

        # objective function
        m.minimize(theta + self.m * m.sum(self.alpha[i]*x[i] for i in range(self.n)))

        # start adding cutting planes
        # start adding cutting planes
        while upper - lower > tol and num_iter < iter_max and time_solve < self.timeout: 

            # get gradient
            dfx = self.get_sub_gradient(xk)


            # update the cutting plane
            m.add_constraint(theta >= m.sum( dfx[i] * (x[i] - xk[i]) for i in range(self.n)) 
                             + self.get_value(xk)
                             )
            
            # solve the new model
            m.solve()
            sol = m.solution 

            # get new vector
            xk = np.array(sol.get_value_list(x))
            
            # update lower bound
            lower = m.objective_value

            # update upperbound
            upper = self.m * sum(self.alpha[i]*xk[i] for i in range(self.n)) + self.get_value(xk)

            # update steps
            num_iter += 1
            
            # update timesolve
            time_solve += m.solve_details.time

            
        optimal = 1 if upper < tol else 0

        return upper, m.objective_value, time_solve, num_iter, optimal
            


# Algorithms testing

In [137]:
data = DataGeneration(20,200,50,1)
solution = Algorithms(data)

In [138]:
solution.cutting_plane()

(8124495.811474778, 8124495.811474779, 2.353241205215454, 74, 0)

In [139]:
solution.linearization()

(8124495.811202649, 4.164870023727417, 0.0)